# Upfront Childcare Provider Scraping Program
This notebook outlines the usage of several scripts dedicated to scraping, parsing, transforming, and cleaning of data from New York State, and New York City public websites. 

### Obtaining Data from Office of Childcare and Family Services:
---
This data is found @ https://ocfs.ny.gov/programs/childcare/looking/ccfs-search.php
- We can utilize the main script found in the OCFS directory. This will gather all of the data from the site and prep it for our cleaning. 
- This can be run with the following command (make sure you have done the steps for correct setup beforehand)
- Be aware, this script takes several hours to run so that it does not spam the website servers. (~5 hours as of December 2024)

In [ ]:
# Runs the OCFS scraping script:
!python OCFS/main.py

We now have our result data, which can be found in ```OCFS/result_data/OCFS_result_data.csv```. This data adhears to the Upfront data model for childcare providers, but there are still some cleaning and validation steps we can take to enhance the quality of the data. For this, we can use PETL and a few helper functions: 

In [2]:
import json
import re
import unicodedata

# This is a JSON hash map of the zip codes per county in NYC. 
with open("nyc_zip_to_county.json", "r") as file:
    zip_to_county = json.load(file)
    zip_to_county[''] = ''

# A hash map of the county name to the borough name. 
county_to_borough = {
    "New York County": "Manhattan",
    "Kings County": "Brooklyn",
    "Queens County": "Queens",
    "Bronx County": "The Bronx",
    "Richmond County": "Staten Island",
    "" : ""
}

# This function standardizes the format of all phone numbers to (111) 222-3333
def standardize_phone_number(phone: str) -> str:
    # Extract all digits from the input string
    digits = re.sub(r'\D', '', phone)
    
    # Validate we have exactly 10 digits
    if len(digits) != 10:
        raise ValueError(f"Phone number must contain exactly 10 digits. Got: {len(digits)} digits")
    
    # Format the digits into the desired pattern
    formatted = f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
    
    return formatted

# This standardizes our faciity names in accordance with Upfronts naming conventions using regular expressions
def standardize_facility_name(name: str, street_name: str = "") -> str:
    name = ''.join(c for c in unicodedata.normalize('NFD', name) if unicodedata.category(c) != 'Mn')
    name = re.sub(r'\b[A-Z]\b', '', name)
    name = name.replace('&', 'and')
    name = name.replace('@', 'at')
    name = re.sub(r',?\s*(LLC|Inc|Incorporated)(\.|$)', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\bYmca\b', 'YMCA', name, flags=re.IGNORECASE)
    name = re.sub(r'\b(?:dba|d\.b\.a\.)\s+', '', name, flags=re.IGNORECASE)
    if street_name:
        name = f"{name.strip()} - {street_name.strip()}"
    name = re.sub(r'\((.*?)\)', r'- \1', name)
    name = re.sub(r'\boperated\b.*$', '', name, flags=re.IGNORECASE).strip()
    replacements = {
        'Ctr\.': 'Center',
        '\bRD\b': 'Road',
        '\bSt\b': 'Street',
        '\bAVE\b': 'Avenue',
        '\bDr\b': 'Drive'
    }
    for abbrev, full in replacements.items():
        name = re.sub(abbrev, full, name, flags=re.IGNORECASE)
    name = re.sub(r'\bIi\b', 'II', name, flags=re.IGNORECASE)
    small_words = {'a', 'an', 'the', 'or', 'of', 'and', 'in', 'on', 'at'}
    name_parts = name.split()
    name = ' '.join(word.lower() if word.lower() in small_words else word for word in name_parts)
    name = re.sub(r'\s*-\s*', ' - ', name)
    name = re.sub(r'#\d+', '', name)
    if street_name:
        name = f"{name.strip()} - {street_name.strip()}"

    return name.strip()


In [3]:
import petl as etl
from distutils.util import strtobool

# Grab the data from the results, and normalize the data within it
OCFS_data = (
    etl
    .fromcsv("OCFS/result_data/OCFS_result_data.csv")    
    .cutout("ADDRESS_BOUROUGH", "ADDRESS_COUNTY")
    .addfield("ADDRESS_COUNTY", lambda row : zip_to_county.get(row.ADDRESS_ZIPCODE[:5]))
    .addfield("ADDRESS_BOUROUGH", lambda row : county_to_borough.get(row.ADDRESS_COUNTY))
    .addfield("AGE_INFANT_MINIMUM", lambda row : "6 Weeks" if bool(strtobool(row.AGE_RANGE_INFANTS)) == True else "")
    .convert("PROGRAM_NAME", lambda v : standardize_facility_name(v))
    .convert("GEN_PHONE_1", lambda v : standardize_phone_number(v))
    .convert("ADDRESS_CITY", lambda v : "New York City")
    )



Sometimes in the data, we have duplicate program names due to the fact that a program might have several locations. If this is the case, we want to add the address location to the name of the program to distinguish it from the other locations. We can do this using a list of duplicate program names and PETL:

In [6]:

# Grab a list of duplicate program names 
OCFS_duplicate_names = list(OCFS_data.duplicates('PROGRAM_NAME').values("PROGRAM_NAME"))

# Change any of the duplicate names so that the program name has the address appended to it if there are more than one location
OCFS_data_deduped = (
    OCFS_data
    .addfield("FIXED_PROGRAM_NAME", lambda row : row.PROGRAM_NAME if not row.PROGRAM_NAME in OCFS_duplicate_names else f"{row.PROGRAM_NAME} - {row.ADDRESS_STREET}")
    .cutout("PROGRAM_NAME")
    .rename("FIXED_PROGRAM_NAME", "PROGRAM_NAME")
)

### Obtaining Data from New York City Health:
---
This data is found @ https://a816-healthpsi.nyc.gov/ChildCare/search.action
- Again, we can utlize our main script for the NYCH directory

In [ ]:
# Runs the NYCH scraping script:
!python NYCH/main.py

We now have the result data for the NCYH site that can be found at ```NYCH/result_data/NYCH_result_data.csv```. This data is also in the Upfront format, and we can also standardize and deduplicate it as we did with the OCFS data:

In [11]:
NYCH_data = (
    etl
    .fromcsv("NYCH/result_data/NYCH_result_data.csv")
    .convert("GEN_WEBSITE", lambda rec : "https://a816-healthpsi.nyc.gov/ChildCare/search") # Hard code this due to it being the same for every record
    .convert("ADDRESS_ZIPCODE", lambda v : v.strip())    
    .cutout("ADDRESS_BOUROUGH", "ADDRESS_COUNTY", "AGE_INFANT_MINIMUM")
    .addfield("ADDRESS_COUNTY", lambda row : zip_to_county.get(row.ADDRESS_ZIPCODE[:5]))
    .addfield("ADDRESS_BOUROUGH", lambda row : county_to_borough.get(row.ADDRESS_COUNTY))
    .addfield("AGE_INFANT_MINIMUM", lambda row : "0 Weeks" if bool(strtobool(row.AGE_RANGE_INFANTS)) == True else "") # Make sure to standarzie on a 0 week minimum if it is in the infant age range
    .convert("PROGRAM_NAME", lambda v : standardize_facility_name(v))
    .convert("GEN_PHONE_1", lambda v : standardize_phone_number(v))
    .convert("ADDRESS_CITY", lambda v : "New York City")
    )

NYCH_duplicate_names = list(NYCH_data.duplicates('PROGRAM_NAME').values("PROGRAM_NAME"))

NYCH_data_deduped = (
    NYCH_data
    .addfield("FIXED_PROGRAM_NAME", lambda row : row.PROGRAM_NAME if not row.PROGRAM_NAME in NYCH_duplicate_names else f"{row.PROGRAM_NAME} - {row.ADDRESS_STREET}")
    .cutout("PROGRAM_NAME")
    .rename("FIXED_PROGRAM_NAME", "PROGRAM_NAME")
)

### Building Final Dataset:
--- 
Now that we have all of our data from both OCFS and NYCH, we can combine the data to get our final dataset of all the available data:

In [13]:
# Re-order the columns:
column_order = [
    "PROGRAM_NAME", "ADDRESS_CITY", "ADDRESS_COUNTRY", "ADDRESS_BOUROUGH", 
    "ADDRESS_COUNTY", "ADDRESS_LATITUDE", "ADDRESS_LONGITUDE", "ADDRESS_STATE", 
    "ADDRESS_STREET", "ADDRESS_ZIPCODE", "AGE_INFANT_MINIMUM", "AGE_RANGE", 
    "AGE_RANGE_1_YEAR", "AGE_RANGE_2_YEARS", "AGE_RANGE_3_YEARS", 
    "AGE_RANGE_4_YEARS", "AGE_RANGE_5_YEARS", "AGE_RANGE_INFANTS", 
    "AGE_RANGE_SCHOOL", "GEN_PHONE_1", "GEN_PROGRAM_SETTING", "GEN_WEBSITE"
]

# Reorder columns for OCFS_data_deduped
OCFS_data_final = OCFS_data_deduped.cut(*column_order)

# Reorder columns for NYCH_data_deduped
NYCH_data_final = NYCH_data_deduped.cut(*column_order)

# Combine both datasets
ALL_results = etl.cat(OCFS_data_final, NYCH_data_final)

# Print the length of all of the result to see how many records we gathered
print(f"Successfully gathered {len(ALL_results)} result records!")

# Save the result CSV:
ALL_results.tocsv("results_final.csv")

Successfully gathered 2704 result records!
